In [ ]:
print('Loading...')

from time import sleep
import json
import jieba
import re
from requests import Session
from bs4 import BeautifulSoup
from pandas import pandas
from collections import Counter
import pickle
from random import random
import nltk
import gensim
from collections import Counter
from gensim import corpora, models, similarities, matutils
from gensim.models import word2vec
from gensim.corpora import WikiCorpus, Dictionary
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
import logging
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import logging
import sys
import csv
import numpy as np
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
import matplotlib.pyplot as plt
from matplotlib import pyplot,font_manager
from matplotlib.font_manager import FontProperties
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

session = Session()

#specify the agent name
#session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'

def crawling():
    
    author_id = input('>>Author id:')
    
    if author_id == '':
        print('No input id found')
    
    else:
        links = []
        prefix_url = 'http://www.ucptt.com'
        for page in range(0,51): #range needs to be specified by case
            base_url = 'http://www.ucptt.com/author/{author_id}/{page}'.format(author_id = author_id, page = page)
            response = session.get(base_url)
            print(f'已經爬到第{page}頁')
            dom = BeautifulSoup(response.text)
            elements = dom.select('div.list-group > a')
            for element in elements:
                link = (prefix_url + element.get('href'))
                links.append(link)
            #sleep(random())
            page += 1

        htmls = []
        for num, link in enumerate(links):
            print(f'還剩下{len(links) - num}個頁面')
            response = session.get(link)
            htmls.append(response.text)
            #sleep(random())

        posts = []
        for html in htmls:
            dom = BeautifulSoup(html)
            for br in dom.find_all("br"):
                br.replace_with('\n')

            title = dom.select('h3')[0].text
            created_time = dom.select('div.panel-heading > span')[0].text
            board = dom.select('ol > li:nth-of-type(2)> a')[0].text
            content = dom.select('div.panel-body')[0].text.replace('\n\n\n\n(adsbygoogle = window.adsbygoogle || []).push({});\n', '') 

            post = dict(title=title, created_time=created_time, board=board, content=content)
            posts.append(post)

            for post in posts:
                post['content'] = re.sub('※ .*\n', '\n', post['content'])
            while re.search('\n: .*\n', post['content']):
                post['content'] = re.sub("\n: .*\n", '\n', post['content'])

        contents = []
        for post in posts:
            content = post['content']
            contents.append(content)

        return contents



def segmentalizing(contents):
    
    jieba.set_dictionary('dict2_PTT.txt')
    with open('stopwords.txt',  encoding="utf-8") as f:
        stopwords = [word.strip() for word in f]
    
    #把文章依脈絡做斷詞
    out = []
    output = ''
    for texts_num, line in enumerate(contents):    
        line = line.strip('\n')
        words = jieba.cut(line, cut_all=False)
        words = [word for word in words if word not in stopwords if word is not ' ' or '\u3000']
        out.append(words)
        #out.extend(words)
    #output = ' '.join(out)
            
    return out


def tokenizing(contents):
    
    jieba.set_dictionary('dict2_PTT.txt')
    with open('stopwords.txt',  encoding='utf-8') as f:
        stopwords = [word.strip() for word in f]
    
    #拿來算詞頻的斷詞
    tokens = []
    for content in contents:
        content = re.sub('\n', '', content)
        content = re.sub('\u3000 ', '', content)
        content = re.sub(' ', '', content)
        content = re.sub(':', '', content)
        token = jieba.lcut(content, cut_all = False)
        tokens.extend(token)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if token is not ' ']
    
    return tokens




def getting_info(contents, tokens, out):
     
	print("本程式提供 5 種資訊：\n")
	print("輸入 1：列出您的所有發文和篇數")
	print("輸入 2：列出您的所有詞彙和詞彙數")
	print("輸入 3：列出您的詞頻表")
	print("輸入 4：計算您的詞匯豐富度")
	print("輸入 5: 測試您的Word2vec模型")
	print("（按control + c退出）")
    
	while True:
    
		try:
			query = str(input('>>輸入：'))
        
			if query == '1':
				print('您共發過' + str(len(contents)) + '篇文章')
				print('----------您的所有發文如下：----------')
				print('\n')
				for content in contents:
					print(content + '\n')
            
			elif query == '2':
				print('您的發文共含有' + str(len(tokens)) + '個詞')
				print('----------您的所有用詞如下：----------')
				print('\n')
				print(' , '.join(tokens))
                
            
			elif query == '3':
				wordcount = Counter(tokens)
				tf = wordcount.most_common(len(wordcount))
				print('即將列出詞頻表———')
				print(tf)
            
            
			elif query == '4':
				types = list(set(tokens))
				vocabulary_richness = len(types) / len(tokens)
				print('\n')
				print('您共使用了' + str(len(tokens)) + '個詞')
				print('您共使用了' + str(len(types)) + '個不同的詞')
				print('\n')
				print('詞彙豐富度為' + str(vocabulary_richness) + '%')
                
			elif query == '5':
				print('--------------Traning model--------------')
				logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
				#sentences = word2vec.LineSentence(out)
				model = word2vec.Word2Vec(out, size=150, window = 8, sg = 1, min_count = 1)
                
				return model
                
			else:
				print('No function found')
            
			print("----------------------------")
        
		except Exception as e:
			print(repr(e))



def w2v(model):

    #logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    #sentences = word2vec.LineSentence(out)
    #model = word2vec.Word2Vec(out, size=150, window = 8, sg = 1, min_count = 1)

    #保存模型，供日後使用
    #model_SG.save("word2vec.model_SG")

    #模型讀取方式
    #model = word2vec.Word2Vec.load("word2vec.model_SG")
    
    print("本程式提供 3 種Word2vec測試模式：\n")
    print("----------輸入一個詞，則去尋找前一百個該詞的相似詞----------")
    print("----------輸入兩個詞，則去計算兩個詞的餘弦相似度----------")
    print("----------輸入三個詞，進行類比推理----------")

    while True:
        try:
            query = input('>>輸入：')
            q_list = query.split()

            if len(q_list) == 1:
                print("相似詞前 100 排序")
                res = model.most_similar(q_list[0],topn = 100)
                for item in res:
                    print(item[0]+","+str(item[1]))

            elif len(q_list) == 2:
                print("計算 Cosine 相似度")
                res = model.similarity(q_list[0],q_list[1])
                print(res)
            
            elif len(q_list) == 3:
                print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
                res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 100)
                for item in res:
                    print(item[0]+","+str(item[1]))
                    
            else:
                print('No function found')
                
            print("--------------------------------------------------------")
            
        except Exception as e:
            print(repr(e))



def main():
    
    print('歡迎來到語料庫建造坊！')
    sleep(random())
    print('本程式專為鄉民打造～')
    sleep(random())
    print('抓取你在PTT上發的文章，幫你建造你的「PTT個人語料庫」')
    sleep(random())
    print('讓你認識網路上真實的自己！')
    sleep(random())
    
#    while True:
        
    try:
        contents = crawling()
        
        if contents:  
            out = segmentalizing(contents)
            tokens = tokenizing(contents)
                
            if out and tokens:
                model = getting_info(contents, tokens, out)
                
                if model:
                    w2v(model)
                        
    except Exception as e:
            print(repr(e))

if __name__ == "__main__":
    main()





Loading...
歡迎來到語料庫建造坊！
本程式專為鄉民打造～
抓取你在PTT上發的文章，幫你建造你的「PTT個人語料庫」
讓你認識網路上真實的自己！
>>Author id:sachia5566
已經爬到第0頁
已經爬到第1頁
還剩下20個頁面
還剩下19個頁面
還剩下18個頁面
還剩下17個頁面
還剩下16個頁面
還剩下15個頁面
還剩下14個頁面
還剩下13個頁面
還剩下12個頁面
還剩下11個頁面
還剩下10個頁面
還剩下9個頁面
還剩下8個頁面
還剩下7個頁面
還剩下6個頁面
還剩下5個頁面
還剩下4個頁面
還剩下3個頁面
還剩下2個頁面
還剩下1個頁面


Building prefix dict from /Users/airmac/Desktop/0321nanoblog/dict2_PTT.txt ...
Loading model from cache /var/folders/1z/q3kr1qp91hdb3582lr3zxlqm0000gn/T/jieba.u613e8ab7a6ae723d71d7015af579ab2e.cache
Loading model cost 0.103 seconds.
Prefix dict has been built succesfully.
Building prefix dict from /Users/airmac/Desktop/0321nanoblog/dict2_PTT.txt ...
Loading model from cache /var/folders/1z/q3kr1qp91hdb3582lr3zxlqm0000gn/T/jieba.u613e8ab7a6ae723d71d7015af579ab2e.cache
Loading model cost 0.087 seconds.
Prefix dict has been built succesfully.


本程式提供 5 種資訊：

輸入 1：列出您的所有發文和篇數
輸入 2：列出您的所有詞彙和詞彙數
輸入 3：列出您的詞頻表
輸入 4：計算您的詞匯豐富度
輸入 5: 測試您的Word2vec模型
（按control + c退出）
>>輸入：1
您共發過20篇文章
----------您的所有發文如下：----------


去年十月以後
各種倒楣的事一直來
3TB的行動硬碟壞了
所有回憶跟珍貴的東西全沒了
電腦壞了
腳踏車輪胎壞了
女友分手了
憂鬱症復發
連前幾天好不容易想買個東西對自己好點
剛買完過兩天就打折
買貴一千塊
感覺真的超差的
我到底是做錯了什麼要遇到這麼多不開心的事
真的很不開心   草泥馬

想妳時
眼淚就流下來
想妳應該是開心的
為什麼現在變的是難過的
是悲傷的
很多事我不懂
我一直在問自己為什麼
大家都說要往前走
可是我找不到往前走的理由
我只知道我每天都睡不著覺
吃藥睡著後醒來頭還是好昏沉
十點上班
但十點十分到辦公室後我就會陷入昏迷
怎麼樣都醒不過來
要到下午一點多我才會恢復清醒
在家我卻怎麼樣都睡不著
每天都很痛苦的生活著
我失去了往前進的勇氣
找不到方向
等下要吃藥了
希望吃了後能好一點
再一小時又要下班了
過一天算一天

抱過很多人
最喜歡抱抱
然而只有妳的擁抱
特別不一樣
妳會緊緊的箍住我
然後用力
掐住我似的
大力的抱好幾下
這是屬於妳的特別
這是只有妳才會這樣的擁抱方式
我好想妳
想妳那樣抱我的時候
第一次
最後一次
打著打著眼眶又濕了
妳要好好的過每一天
想妳了
我在很遠的未來等妳
在另一個平行時空裡我們沒有分手
只不過是妳出國了而已
我會一直等下去
擦乾淚,再一個鐘頭就下班了
又過了一天

電視賣掉後
就在想該是買投影機的時候了
剛剛就直接給他買下去
BenQ的mh545
16900
看了一下到貨時間
居然說今天就會到貨
真的會這摸快嗎？
這晚大概是無法睡了
今天下班後再吃藥好好睡一覺吧
下一個目標是筆電
筆電好像也該換了
買買買
在某個限度下對自己好點吧
這個世界上你只有自己了
你會好起來的
會幸福的

新的住處緊鄰馬路
車來車往滿大聲的
開始找吸音海棉找了一小時
最後還是用一隻手機放輕音樂
然後塞上衛生紙做的耳塞
果然好多了。。。
我真的很怕不好聽的聲音
不好睡
也一直醒
改天要來找耳塞了
到新家的第一個晚上
新的生活開

>>輸入：3
即將列出詞頻表———
[('妳', 72), ('好', 61), ('想', 31), ('說', 27), ('人', 21), ('做', 17), ('只', 15), ('真', 13), ('睡', 13), ('抱', 13), ('走', 12), ('錢', 12), ('後', 11), ('一直', 11), ('下', 11), ('事', 10), ('吃', 10), ('找', 9), ('知道', 9), ('一個', 9), ('問題', 9), ('東西', 8), ('時', 8), ('買', 8), ('話', 8), ('覺得', 8), ('\u3000', 8), ('應該', 7), ('前', 7), ('我會', 7), ('新', 7), ('辦', 7), ('開心', 6), ('一點', 6), ('再', 6), ('住', 6), ('死', 6), ('努力', 6), ('地方', 6), ('20%', 6), ('想念', 6), ('貸款', 6), ('分手', 5), ('生活', 5), ('希望', 5), ('一天', 5), ('未來', 5), ('已', 5), ('一種', 5), ('壞了', 4), ('回憶', 4), ('女友', 4), ('感覺', 4), ('現在', 4), ('卻', 4), ('今天', 4), ('一定', 4), ('變成', 4), ('中', 4), ('太', 4), ('還要', 4), ('比較好', 4), ('曾', 4), ('完全', 4), ('難', 4), ('是不是', 4), ('新毒瘤', 4), ('柯P', 4), ('最', 4), ('長', 4), ('板友', 4), ('一起', 4), ('30%', 4), ('處', 4), ('日子', 4), ('改變', 4), ('夢', 4), ('年', 3), ('幾天', 3), ('底', 3), ('每天', 3), ('家', 3), ('下班', 3), ('世界', 3), ('聲音', 3), ('瘋', 3), ('請', 3), ('搬家', 3), ('好笑', 3), ('龜', 3), ('支持', 

>>輸入：4


您共使用了1741個詞
您共使用了962個不同的詞


詞彙豐富度為0.5525560022975301%
----------------------------
>>輸入：5


...

相似詞前 100 排序
KeyError("word '男女' not in vocabulary",)
>>輸入：5566
相似詞前 100 排序
KeyError("word '5566' not in vocabulary",)
>>輸入：3
相似詞前 100 排序
KeyError("word '3' not in vocabulary",)
>>輸入：台大
相似詞前 100 排序
KeyError("word '台大' not in vocabulary",)
>>輸入：學校
相似詞前 100 排序
KeyError("word '學校' not in vocabulary",)
>>輸入：公司
相似詞前 100 排序
KeyError("word '公司' not in vocabulary",)
>>輸入：想


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


相似詞前 100 排序

,0.9996070861816406
　,0.9994709491729736
說,0.9994660019874573
好,0.9993903636932373
真,0.9993060231208801
問題,0.9993019104003906
睡,0.9992911219596863
人,0.9992713332176208
我會,0.9992656707763672
走,0.9992649555206299
一個,0.9992536902427673
覺得,0.9992533326148987
話,0.9992418885231018
新,0.9992357492446899
後,0.9992232918739319
做,0.9992218017578125
 ,0.9992170929908752
應該,0.9992150664329529
知道,0.9992124438285828
抱,0.999201774597168
妳,0.999163031578064
開心,0.9991612434387207
辦,0.999151349067688
吃,0.9991459846496582
地方,0.9991379976272583
錢,0.9991352558135986
一直,0.9991284012794495
前,0.999122679233551
事,0.9991208910942078
貸款,0.9991172552108765
一起,0.9991087913513184
生活,0.9990977048873901
買,0.9990897178649902
中,0.9990844130516052
努力,0.9990828037261963
一種,0.9990823268890381
死,0.9990695714950562
分手,0.9990543723106384
只,0.9990447163581848
空心,0.9990337491035461
還要,0.9990274310112
找,0.9990248084068298
改變,0.9990190267562866
現在,0.999018132686615
卻,0.9990103244781494
20%,0.9990087747573853
下,0.99900

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:234: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


計算 Cosine 相似度
0.99847347
--------------------------------------------------------
